In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.patches import Circle
from mpl_toolkits.mplot3d import Axes3D

from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache
from allensdk.core.reference_space import ReferenceSpace

In [2]:
sys.path.append(r'C:\Users\lesliec\code')

In [3]:
from tbd_eeg.tbd_eeg.data_analysis.eegutils import EEGexp

In [4]:
%matplotlib notebook

### Load EEG electrode coordinates from EEGexp class
I pulled the coordinates from the image here: https://neuronexus.com/electrode-array/mouse-eeg/. (ML: neg= mouse left, pos= mouse right)

In [5]:
EEGch_coords = EEGexp.EEG_channel_coordinates
EEGch_coords.head()

,AP,ML
0,-4.14,-4.05
1,-4.14,-2.24
2,-4.14,-1.00
3,-3.04,-4.13
4,-3.04,-2.88


### Load CCF

In [6]:
mcc = MouseConnectivityCache(resolution=25)

template, template_info = mcc.get_template_volume()
annot, annot_info = mcc.get_annotation_volume()

structure_tree = mcc.get_structure_tree()

### Convert EEG coordinates to CCF coordinates

Using coordinates from the Paxinos atlas (here: http://labs.gaidi.ca/mouse-brain-atlas/), I am assigning bregma based on the corpus callosum joining between +1.2 and +1.1 mm AP.
<br> Or see https://github.com/cortex-lab/allenCCF/blob/master/Browsing%20Functions/allenCCFbregma.m for the estimate from SharpTrack, which is in 10 um resolution. The equivalent would be AP pixel = 216, quite close to my estimate.

In [7]:
mm_slice = 0.025 # mm, we are using CCF with 25 um resolution
cc_join_ind = 168 # AP_ind, I call this AP +1.1 mm

AP_zero_ind = cc_join_ind + int(1.1 / mm_slice)
print('Bregma (AP) pixel ind: {}'.format(AP_zero_ind))

ML_zero_ind = np.shape(annot)[2]//2
print('Bregma (ML) pixel ind: {}'.format(ML_zero_ind))

## Find brain surface at bregma to find DV pixel ##
edge_indb = np.nonzero(annot[AP_zero_ind, int(np.shape(annot)[1]/2), :])[0][0]
bregma_surfaceML = np.arange(edge_indb, np.shape(annot)[2] - edge_indb)
bregma_surfaceDV = np.zeros_like(bregma_surfaceML)
for k, bsurML in enumerate(bregma_surfaceML):
    bregma_surfaceDV[k] = np.nonzero(annot[int(AP_zero_ind), :, int(bsurML)])[0][0]

DV_zero_ind = np.min(bregma_surfaceDV)
print('Bregma (DV) pixel ind: {}'.format(DV_zero_ind))

Bregma (AP) pixel ind: 212
Bregma (ML) pixel ind: 228
Bregma (DV) pixel ind: 28


Conversions for AP and ML coordinates:

In [8]:
EEGch_coords['x'] = (AP_zero_ind - (EEGch_coords['AP'] / mm_slice)).astype(int)
# neg AP coords -> greater slice ind
EEGch_coords['z'] = (ML_zero_ind + (EEGch_coords['ML'] / mm_slice)).astype(int)
# neg ML coords -> smaller slice ind (left)
EEGch_coords.head()

,AP,ML,x,z
0,-4.14,-4.05,377,66
1,-4.14,-2.24,377,138
2,-4.14,-1.00,377,188
3,-3.04,-4.13,333,62
4,-3.04,-2.88,333,112


### Loop to adjust ML and DV coords of electrode based on curvature

In [9]:
new_ML_pix_inds = np.zeros(len(EEGch_coords))
new_DV_pix_inds = np.zeros(len(EEGch_coords))
for AP_pix_ind, slice_group in EEGch_coords.groupby(['x']):
    
    # get surface of brain
    edge_ind = np.nonzero(annot[int(AP_pix_ind), int(np.shape(annot)[1]/2), :])[0][0]
    surfaceML = np.arange(edge_ind, np.shape(annot)[2] - edge_ind)
    surfaceDV = np.zeros_like(surfaceML)
    for k, surML in enumerate(surfaceML):
        surfaceDV[k] = np.nonzero(annot[int(AP_pix_ind), :, int(surML)])[0][0]
    surface_distance = np.concatenate(([0], np.cumsum(np.sqrt(np.diff(surfaceML)**2 + np.diff(surfaceDV)**2))))
    
    # Left side
    lelec = slice_group[slice_group['ML'] < 0].sort_values(['ML'], ascending=False)
    lseedML = lelec['ML'].iloc[0]
    lseedMLpix = lelec['z'].iloc[0]
    for index, row in lelec.iterrows():
        if row['z'] == lseedMLpix:
            new_ML_pix_inds[index] = lseedMLpix
            new_DV_pix_inds[index] = surfaceDV[surfaceML == lseedMLpix][0]
        else:
            seeddist = (lseedML - row['ML']) / mm_slice
            newind = np.argmin(np.abs((surface_distance - surface_distance[surfaceML == lseedMLpix]) + seeddist))
            new_ML_pix_inds[index] = surfaceML[newind]
            new_DV_pix_inds[index] = surfaceDV[newind]
    
    # Right side
    relec = slice_group[slice_group['ML'] > 0].sort_values(['ML'])
    rseedML = relec['ML'].iloc[0]
    rseedMLpix = relec['z'].iloc[0]
    for index, row in relec.iterrows():
        if row['z'] == rseedMLpix:
            new_ML_pix_inds[index] = rseedMLpix
            new_DV_pix_inds[index] = surfaceDV[surfaceML == rseedMLpix][0]
        else:
            seeddist = (rseedML - row['ML']) / mm_slice
            newind = np.argmin(np.abs((surface_distance - surface_distance[surfaceML == rseedMLpix]) + seeddist))
            new_ML_pix_inds[index] = surfaceML[newind]
            new_DV_pix_inds[index] = surfaceDV[newind]

In [10]:
EEGch_coords['y'] = new_DV_pix_inds.astype(int)
EEGch_coords['z'] = new_ML_pix_inds.astype(int)
EEGch_coords.head()

,AP,ML,x,z,y
0,-4.14,-4.05,377,82,58
1,-4.14,-2.24,377,141,24
2,-4.14,-1.00,377,188,20
3,-3.04,-4.13,333,79,47
4,-3.04,-2.88,333,119,25


### Add CCF structure acronym

In [11]:
# create empty lists to hold info (will add it to EEGch_coords dataframe after loop)
structure_id = []
structure_acronym = []
structure_name = []
parent_id = []
parent_name = []
structure_parent_acronym = []

# for loop through the EEGch_coords dataframe
for indi, row in EEGch_coords.iterrows():
    # choose first non-zero value in the annotated array at the AP, ML pixel inds
    non_zero_DV = np.nonzero(annot[int(row.x),:,int(row.z)])[0][0]
    struct_id_temp = annot[int(row.x), non_zero_DV, int(row.z)]
    # append structure id, name, parent info to empty lists
    structure_id.append(struct_id_temp)
    structure_acronym.append(structure_tree.get_structures_by_id([struct_id_temp])[0]['acronym'])
    structure_name.append(structure_tree.get_structures_by_id([struct_id_temp])[0]['name'])
    parent_id.append(structure_tree.parents([struct_id_temp])[0]['id'])
    parent_name.append(structure_tree.parents([struct_id_temp])[0]['name'])
    structure_parent_acronym.append(structure_tree.parents([struct_id_temp])[0]['acronym'])
    
# add lists created to original dataframe
EEGch_coords['location'] = structure_parent_acronym
EEGch_coords['structure_id'] = structure_id
EEGch_coords['structure_acronym'] = structure_acronym
EEGch_coords['structure_name'] = structure_name
EEGch_coords['parent_id'] = parent_id
EEGch_coords['parent_name'] = parent_name
EEGch_coords['parent_acronym'] = structure_parent_acronym
EEGch_coords.head()

,AP,ML,x,z,y,location,structure_id,structure_acronym,structure_name,parent_id,parent_name,parent_acronym
0,-4.14,-4.05,377,82,58,VISl,421,VISl1,"Lateral visual area, layer 1",409,Lateral visual area,VISl
1,-4.14,-2.24,377,141,24,VISp,593,VISp1,"Primary visual area, layer 1",385,Primary visual area,VISp
2,-4.14,-1.00,377,188,20,RSPagl,671,RSPagl1,"Retrosplenial area, lateral agranular part, la...",894,"Retrosplenial area, lateral agranular part",RSPagl
3,-3.04,-4.13,333,79,47,VISal,1074,VISal1,"Anterolateral visual area, layer 1",402,Anterolateral visual area,VISal
4,-3.04,-2.88,333,119,25,VISp,593,VISp1,"Primary visual area, layer 1",385,Primary visual area,VISp


In [18]:
dvvals = EEGch_coords['DV_pixel_ind'].values
print(dvvals)
print(np.min(dvvals))

[58 24 20 47 25  7 47 26  9 51 29 51 43 63 63 63 64 43 51 29 51  9 25 47
  7 24 47 20 24 58]
7


## Save as a .csv

In [13]:
EEG_electrodes_filename = os.path.join(r'E:\NWB_testing', r'EEG_electrodes_info.csv')

In [14]:
EEGch_coords.to_csv(EEG_electrodes_filename, index=False)